In [1]:
import bpy
from bpy.props import (
    FloatProperty,
    PointerProperty,
)
import numpy as np

Device with name Apple M1 Pro supports metal minimum requirements


In [2]:
#open blender file
bpy.ops.wm.open_mainfile(filepath="tokamak.blend")

Read blend: "/Users/arnaudbergeron/Desktop/Code/2023/WaveViz/tokamak.blend"


{'FINISHED'}

In [4]:
tokamak_obj = bpy.data.objects['tokamamamam']

In [6]:
to_anim = np.load('to_anim.npy')

In [7]:
obj = (tokamak_obj.data.color_attributes[0])

for i_frame in range(1, to_anim.shape[0]):
    new_att = (tokamak_obj.data.color_attributes.new(name=f'col{i_frame}', type='FLOAT_COLOR', domain='POINT'))
    for idx, new_v in enumerate(new_att.data):
        new_v.color = to_anim[i_frame][idx][3:]

In [8]:
material = bpy.data.materials['Verts']
link_node = None

mix_nodes = []
old_node = (material.node_tree).nodes['Attribute']

for i_frame in range(1, to_anim.shape[0]):
    mix_node = material.node_tree.nodes.new(type='ShaderNodeMixRGB')
    mix_node.blend_type = 'MIX'
    mix_node.inputs['Fac'].default_value = 0.0

    new_node = material.node_tree.nodes.new(type='ShaderNodeAttribute')
    new_node.attribute_name = f'col{i_frame}'

    #link old_node and new_node alpha to mix_node color
    try:
        material.node_tree.links.new(old_node.outputs['Alpha'], mix_node.inputs['Color1'])
    except KeyError:
        material.node_tree.links.new(old_node.outputs['Color'], mix_node.inputs['Color1'])

    material.node_tree.links.new(new_node.outputs['Alpha'], mix_node.inputs['Color2'])

    old_node = mix_node
    mix_nodes.append(mix_node)

material.node_tree.links.new(old_node.outputs['Color'], (material.node_tree).nodes['Principled BSDF'].inputs['Alpha'])

bpy.data.materials['Verts'].node_tree

In [9]:
#keyframe the fac to 1
for idx, node in enumerate(mix_nodes):
    node.inputs['Fac'].keyframe_insert(data_path='default_value', frame=idx)
    node.inputs['Fac'].default_value = 1.0
    node.inputs['Fac'].keyframe_insert(data_path='default_value', frame=idx+1)

In [10]:
bpy.ops.wm.save_as_mainfile(filepath='tokamak_2.blend')

Info: Saved "tokamak_2.blend"


{'FINISHED'}